In [7]:
from utils import *
import matplotlib
import numpy as np

location_id = 0 #JSON with all coordinates has multiple locations (18), pick which one to work with

num_splits_for_location = 20 #in how many splits to divide the location (20 = 20x20 grid, 100 = 100x100 grid, etc.)



In [8]:
json_path = r'data/catchments_long.json' #path to JSON file with all coordinates

#parse JSON to list of coordinates (dictionaries). See function definition for long_json_to_list()
#extract one dict form the list at id: location_id
#divide the location into a grid of num_splits_for_location x num_splits_for_location. done by corners_to_grid_by_num_splits()
#location_grid_by_num_splits: 2 locumns - latitude and longitude, num_splits_for_location^2 rows - coordinates of the grid
location_grid_by_num_splits = corners_to_grid_by_num_splits(long_json_to_list(json_path)[location_id], num_splits_for_location)

#call the elevation api of every lat,lon pair in the location_grid_by_num_splits. elevation_api_caller()
#parse the response to a dataframe. parse_api_resonse_to_dataframe()
#pivot the dataframe to a grid. pivot()
pivoted_elevation_grid_by_num_splits = parse_api_resonse_to_dataframe(elevation_api_caller(location_grid_by_num_splits)).pivot(index='latitude', columns='longitude', values='elevation')

In [9]:
catchemnts_dict_list = find_catchments_from_df(df = pivoted_elevation_grid_by_num_splits, radius= 1)

In [13]:
colors = []
for i in range(len(catchemnts_dict_list)):
    colors.append(f'hsv({i/len(catchemnts_dict_list)},100%,100%)')

In [33]:
#for plotting local minima
import plotly.graph_objects as go
import numpy as np

test_group = catchemnts_dict_list[14]
np_test_group = np.array(test_group['cathcment_points'])
np_test_group_lats = [pivoted_elevation_grid_by_num_splits.index[item] for item in np_test_group[:,0]]
np_test_group_lons = [pivoted_elevation_grid_by_num_splits.columns[item] for item in np_test_group[:,1]]
np_test_group_elevations = [pivoted_elevation_grid_by_num_splits.iloc[item[0], item[1]] for item in np_test_group]


# Plot our elevation data, 
fig = go.Figure(data=[go.Surface(z=pivoted_elevation_grid_by_num_splits.transpose().values, x=pivoted_elevation_grid_by_num_splits.index, y=pivoted_elevation_grid_by_num_splits.columns)])
# fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5, color='limegreen', symbol='circle'), name='Scatterplot surface'))  # plots all points in scatterplot form

fig.add_trace(go.Scatter3d(x=np_test_group_lats, y=np_test_group_lons, z=np_test_group_elevations, mode='markers', marker=dict(size=5, color=colors[0], symbol='circle'), name='catchment points'))

fig.add_trace(go.Scatter3d(x=[pivoted_elevation_grid_by_num_splits.index[test_group['minima'][0]]], y=[pivoted_elevation_grid_by_num_splits.columns[test_group['minima'][1]]], z=np.array(pivoted_elevation_grid_by_num_splits.iloc[test_group['minima'][0], test_group['minima'][1]]), mode='markers', marker=dict(size=5, color='red', symbol='circle'), name='Local Minima'))


fig.update_layout(
    scene=dict(
        aspectratio=dict(x=1, y=1, z=0.2),
        aspectmode='manual'
    )
)

fig.show()

In [61]:
#for plotting local minima
import plotly.graph_objects as go
import numpy as np

scatter_raise_by = 100
scatter_size = 3
surface_opacity = 0.8
surface_colorscale = 'gray' #default viridis, other options: https://plotly.com/python/builtin-colorscales/
traces = []

traces.append(go.Surface(z=pivoted_elevation_grid_by_num_splits.transpose().values, 
                         x=pivoted_elevation_grid_by_num_splits.index, 
                         y=pivoted_elevation_grid_by_num_splits.columns,
                         opacity=surface_opacity,
                         colorscale=surface_colorscale,))

for i in range(len(catchemnts_dict_list)):
    #color = f'hsv({(i/len(catchemnts_dict_list))*255},100,100)'
    #color = go.Color((i+1)/len(catchemnts_dict_list), s=1, v=1)
    color = i
    color = f'rgb({np.random.randint(0,255)},{np.random.randint(0,255)},{np.random.randint(0,255)})'

    test_group = catchemnts_dict_list[i]
    np_test_group = np.array(test_group['cathcment_points'])
    np_test_group_lats = [pivoted_elevation_grid_by_num_splits.index[item] for item in np_test_group[:,0]]
    np_test_group_lons = [pivoted_elevation_grid_by_num_splits.columns[item] for item in np_test_group[:,1]]
    np_test_group_elevations = [pivoted_elevation_grid_by_num_splits.iloc[item[0], item[1]]+scatter_raise_by for item in np_test_group]

    traces.append(go.Scatter3d(x=np_test_group_lats, 
                               y=np_test_group_lons, 
                               z=np_test_group_elevations, 
                               mode='markers', 
                               marker=dict(size=scatter_size, color=color, symbol='circle')))

    traces.append(go.Scatter3d(x=[pivoted_elevation_grid_by_num_splits.index[test_group['minima'][0]]], 
                               y=[pivoted_elevation_grid_by_num_splits.columns[test_group['minima'][1]]], 
                               z=np.array(pivoted_elevation_grid_by_num_splits.iloc[test_group['minima'][0], test_group['minima'][1]]+scatter_raise_by), 
                               mode='markers', 
                               marker=dict(size=scatter_size, color=color, symbol='diamond')))

fig = go.Figure(data=traces)
fig.update_layout(
    scene=dict(
        aspectratio=dict(x=1, y=1, z=0.2),
        aspectmode='manual'
    ), 
    width=800,
    height=800
)

fig.show()